In [1]:
import os
import cv2
import importlib
import torch
import numpy as np
import argparse
import yaml
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageOps
import json
import nltk
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import clip
import kornia
import torchvision
import skimage.color
import random

from utils_func import *
from html_images import *

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--device', type=str, default='cuda:0')

parser.add_argument('--root_dir', type=str, default='C:/MyFiles/CondTran/finals/bert_final')
parser.add_argument('--log_dir', type=str, default='logs/bert')
parser.add_argument('--step', type=str, default='142124')

# Load transformer
args = parser.parse_args(args=[])
os.chdir(args.root_dir)
sys.path.append(args.root_dir)
module = importlib.import_module(f'filltran.models.colorization')
args.fill_model = getattr(module, 'Colorization')
filltran = load_model(args.fill_model, args.log_dir, args.step).to(args.device).eval().requires_grad_(False)

def rgb_to_L(img):
    img = np.array(img)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l = lab[:, :, 0]
    return Image.fromarray(l)

def resize_color(l, rgb):
    rgb = np.array( rgb.resize(l.size) )
    l = np.array(l)
    l = np.expand_dims(l, axis=2)
    ab = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)[:, :, 1:3]
    
    lab = np.concatenate([l, ab], axis=2)
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    return Image.fromarray(img)

Working with z of shape (1, 1024, 16, 16) = 262144 dimensions.


In [14]:
img_path = 'C:\\MyFiles\\CondTran\\data\\raw\\in1.JPEG'
I_color = Image.open(img_path).convert('RGB')
I_gray = I_color.convert('L')
I_gray.save('C:\\MyFiles\\CondTran\\sample\\results\\gray.png')
I_color.save('C:\\MyFiles\\CondTran\\sample\\results\\original.png')

x_color = preprocess(I_color, None).to(args.device)
x_gray = preprocess(I_gray, None).to(args.device)

idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape(idx_color.shape[0], f_gray.shape[2], f_gray.shape[3])
#idx_color = torch.randint(low=0, high=4097, size=idx_color.shape).long().to(args.device)
x_rec = filltran.hybrid_vqgan.decode(idx_color, f_gray)
I_rec = output_to_pil(x_rec[0])
I_rec.save('C:\\MyFiles\\CondTran\\sample\\results\\recon.png')

x_color[:, 0, :, :] = 0.0; x_color[:, 1, :, :] = -0.6; x_color[:, 2, :, :] = -0.6
idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape(idx_color.shape[0], f_gray.shape[2], f_gray.shape[3])
#idx_color = torch.randint(low=0, high=4097, size=idx_color.shape).long().to(args.device)
x_rec = filltran.hybrid_vqgan.decode(idx_color, f_gray)
output_to_pil(x_rec[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\recon_r.png')
output_to_pil(x_color[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\r.png')

x_color[:, 0, :, :] = -0.6; x_color[:, 1, :, :] = 0.0; x_color[:, 2, :, :] = -0.6
idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape(idx_color.shape[0], f_gray.shape[2], f_gray.shape[3])
#idx_color = torch.randint(low=0, high=4097, size=idx_color.shape).long().to(args.device)
x_rec = filltran.hybrid_vqgan.decode(idx_color, f_gray)
output_to_pil(x_rec[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\recon_g.png')
output_to_pil(x_color[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\g.png')

x_color[:, 0, :, :] = -0.6; x_color[:, 1, :, :] = -0.6; x_color[:, 2, :, :] = 0.0
idx_color, f_gray = filltran.hybrid_vqgan.encode(x_color, x_gray)
idx_color = idx_color.reshape(idx_color.shape[0], f_gray.shape[2], f_gray.shape[3])
#idx_color = torch.randint(low=0, high=4097, size=idx_color.shape).long().to(args.device)
x_rec = filltran.hybrid_vqgan.decode(idx_color, f_gray)
output_to_pil(x_rec[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\recon_b.png')
output_to_pil(x_color[0]).save('C:\\MyFiles\\CondTran\\sample\\results\\b.png')

In [11]:
img_path = 'C:\\MyFiles\\CondTran\\data\\stroke\\3.JPEG'
I_color = Image.open(img_path).convert('RGB')
I_gray = I_color.convert('L')
I_l = rgb_to_L(I_color)

diff = np.array(I_gray).astype(int) - np.array(I_l).astype(int)

print(np.max(diff))

-58


In [14]:
img_path = 'C:\\MyFiles\\CondTran\\data\\stroke\\3.JPEG'
I_color = Image.open(img_path).convert('RGB')
I_gray = I_color.convert('L')

I_l = rgb_to_L(I_gray.convert('RGB'))

diff = np.array(I_gray).astype(int) - np.array(I_l).astype(int)

I_gray.show()
I_l.show()

print(np.min(diff))

-9
